<a href="https://colab.research.google.com/github/giovannibaratta/GTSRB/blob/master/ModelTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Import e definizioni (Versione 5) { vertical-output: true, display-mode: "form" }
#@markdown Selezionare se si vuole caricare una versione specifica di tensorflow
forceVersion = False #@param {type:"boolean"} 
tfVersion = "1.13.1" #@param ["1.12.2", "1.13.1", "1.14.0rc1", "1.14.0", "2.0.0b1", "PRINT_AV_VERSION"] {allow-input: true}
#@markdown Selezionare se si è collegati ad un runtime locale (non quello di colab)
localRuntime = False #@param{type:"boolean"}
#@markdown Selezionare se si intende utilizzare le TPU come acceleratore (solo se il runtime è quello di colab)
useTPU = False #@param {type:"boolean"}

useTPU = False if localRuntime else useTPU

if forceVersion == True:
  if useTPU == True:
    !pip install -q tensorflow=={tfVersion}
  else:
    !pip install -q tensorflow-gpu=={tfVersion}

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pprint
import math
import random as rn
#data aug
from tensorflow import keras 
# gestione directory
import shutil
import sys
# colorare output
!pip install -q colorama
from colorama import Style as ColoramaStyle
import re
from datetime import datetime

currentPath = !pwd
#@markdown Path da utilizzare nel caso di runtime remoto
colabPath = '/gdrive/My Drive/DatasetSegnaliStradali/GTSRB/' #@param{type:'string'} 
#@markdown Path da utilizzare nel caso di runtime locale
localPath =  "/Desktop/DL/" #@param{type:'string'} 
completeLocalPath = currentPath[0] + localPath
#@markdown Cartella da utilizzare all'interno del path per l'output dei risultati (la cartella deve essere già presente)
phase = "trainingPhase1" #@param ["trainingPhase1", "trainingPhaseCNN", "trainingPhaseResNet", "trainingPhaseInception", "trainingPhaseGroupModel"] {allow-input: true}
rootPath = completeLocalPath if localRuntime else colabPath

#@markdown Dimensioni delle immagini di training, validation e test
width = 48#@param{type:'integer'}
height = 48#@param{type:'integer'}
tmpPath = rootPath + 'tmp/'
testDir = rootPath + 'data/test' + str(width) + 'x' + str(height) + '/'

print('Versione TF : ', ColoramaStyle.BRIGHT, tf.__version__, ColoramaStyle.RESET_ALL,sep="")

if useTPU == False:
  if 'COLAB_TPU_ADDR' in os.environ:
    raise RuntimeError("Hai selezionato il runtime TPU. Cambia l'impostazioni del notebook")
  print("Acceleratore : GPU")
  print("Verifica device :",tf.test.gpu_device_name())
  tpu_address = ''
else:
  if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
  else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print("Acceleratore : TPU")
    print ('Verifica device : ', tpu_address)

if not localRuntime:
  from google.colab import drive
  from google.colab import files
  drive.mount('/gdrive')
    
#import script da gdrive
scriptPath = rootPath + "scripts/colab/"
sys.path.append(scriptPath)

from CommonUtils import *
from ModelBuilderUtils import *
from TrainingUtils import *
from TestUtils import *

if tfVersion()['MAJOR'] < 2:
  tf.logging.set_verbosity(tf.logging.ERROR)
resetSeed()

groupInfo = {
    0 : 'prohibitory',
    1 : 'danger',
    2 : 'mandatory',
    3 : 'others',
    4 : 'limitsEnd'
}

labelsInfo = {
    # labelNumber : (group,labelInsideGroup, description)
    0 : (0,0, '20 km/h'),
    1 : (0,1, '30 km/h'),
    2 : (0,2, '50 km/h'),
    3 : (0,3, '60 km/h'),
    4 : (0,4, '70 km/h'),
    5 : (0,5, '80 km/h'),
    6 : (4,0, '80 km/h end'),
    7 : (0,6, '100 km/h'),
    8 : (0,7, '120 km/h'),
    9 : (0,8, 'No overtaking'),
    10 : (0,9, 'No overtaking for tracks'),
    11 : (1,0, 'Crossroad with secondary way'),
    12 : (3,0, 'Main road'),
    13 : (3,1,  'Give way'),
    14 : (3,2, 'Stop'),
    15 : (0,10,  'Road up'),
    16 : (0,11, 'Road up for track'),
    17 : (3,3,  'Brock'),
    18 : (1,1, 'Other dangerous'),
    19 : (1,2, 'Turn left'),
    20 : (1,3, 'Turn right'),
    21 : (1,4, 'Winding road'),
    22 : (1,5, 'Hollow road'),
    23 : (1,6, 'Slippery road'),
    24 : (1,7, 'Narrowing road'),
    25 : (1,8, 'Roadwork'),
    26 : (1,9, 'Traffic light'),
    27 : (1,10, 'Pedestrian'),
    28 : (1,11, 'Children'),
    29 : (1,12, 'Bike'),
    30 : (1,13, 'Snow'),
    31 : (1,14, 'Deer'),
    32 : (4,1, 'End of the limits'),
    33 : (2,0, 'Only right'),
    34 : (2,1, 'Only left'),
    35 : (2,2, 'Only straight'),
    36 : (2,3, 'Only straight and right'),
    37 : (2,4, 'Only straight and left'),
    38 : (2,5, 'Take right'),
    39 : (2,6, 'Take left'),
    40 : (2,7, 'Circle crossroad'),
    41 : (4,2, 'End of overtaking limit'),
    42 : (4,3, 'End of overtaking limit for track')
}

lookUpTable=[
    # gruppo 0
    {
        0:0,
        1:1,
        2:2,
        3:3,
        4:4,
        5:5,
        6:7,
        7:8,
        8:9,
        9:10,
        10:15,
        11:16
    },
    # gruppo 1
    {
        0:11,
        1:18,
        2:19,
        3:20,
        4:21,
        5:22,
        6:23,
        7:24,
        8:25,
        9:26,
        10:27,
        11:28,
        12:29,
        13:30,
        14:31
    },
    # gruppo 2
    {
        0:33,
        1:34,
        2:35,
        3:36,
        4:37,
        5:38,
        6:39,
        7:40
    },
    # gruppo 3
    {
        0:12,
        1:13,
        2:14,
        3:17
    },
    #gruppo 4
    {
        0:6,
        1:32,
        2:41,
        3:42  
    }
]

In [0]:
#@title Model testing { vertical-output: true }
from ipywidgets import *
from IPython.display import display
import notebook

def sortByTime(elem):
    sec = int(elem.split("_")[-1])
    minutes = int(elem.split("_")[-2])
    hours = int(elem.split("_")[-3])
    day = int(elem.split("_")[-4])
    month = int(elem.split("_")[-5])
    year = int(elem.split("_")[-6])
    return datetime(year, month, day, hours, minutes, sec)

def geLosstWeightsFromPath(path):
  weights = list(filter(lambda name:name.startswith("epoch"),os.listdir(path)))
  if len(weights) != 1:
    raise ValueError("nella directory" + path + "sono presenti più o zero pesi")
  loss = float(weights[0].split("_")[-1].replace(".hdf5",""))
  return (loss, path + weights[0])

printPlot = False #@param {type:"boolean"}

# larghezza colonna
gridWidth = 400 #@param {type:"slider", min:100, max:400, step:20}
# altezza riga 
gridHeight = 20 #@param {type:"slider", min:15, max:30, step:5}
# numero di colonne nella griglia
repeat = 2 #@param {type:"slider", min:1, max:5, step:1}
# imposta automaticamente l'ultimo dello a True
autoSelectLast = True
dateRegex = '[0-9]{4}_[1]{0,1}[0-9]_[1-3]{0,1}[0-9]_[1,2]{0,1}[0-9]_[1-6]{0,1}[0-9]_[1-6]{0,1}[0-9]'
testData, padding = loadTestData(testDir, width, height)
labels = np.load(testDir + "labels.npy")
trainingPath = rootPath + phase + "/"

output = widgets.Output()

predictions = None
modelsName = None
joinCheckboxes = None


def onUpdateClicked(button):
    global predictions
    global modelsName
    global joinCheckboxes
    with output:
        output.clear_output()
        print("Calcolo accuracy su Test set dei modelli selezionati")
        modelsAndWeights = []
        modelsName = [chb.description for chb in checkboxes if chb.value == True]
        for index in range(len(modelsName)):
            modelName = modelsName[index]
            modelPath = trainingPath + modelName + "/"
            lrList = list(filter(lambda name:re.search('^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$',name),os.listdir(modelPath)))
            minLoss = 999999
            weightsFileName = None
            for lr in lrList:
                #controllo nel main
                mainPath = modelPath + lr + "/Main/weights/"
                loss, weightsFile = geLosstWeightsFromPath(mainPath)
                if loss < minLoss:
                    minLoss = loss
                    weightsFileName = weightsFile
                #se è presente il fineTuning controllo anche quello
                fineTuningPath = modelPath + lr + "/FineTuning/"
                if os.path.exists(fineTuningPath):
                    fineTuningLrList = list(filter(lambda name:re.search('^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$',name),os.listdir(fineTuningPath)))
                    for ftLR in fineTuningLrList:
                        loss, weightsFile = geLosstWeightsFromPath(fineTuningPath + ftLR + "/weights/")
                        if loss < minLoss:
                            minLoss = loss
                            weightsFileName = weightsFile
            if weightsFile == None:
                raise ValueError("Errore durante la ricerca del miglior training")
            
            # includere le funzioni per trasformare un modello json in un modello normale
            # e in base alla versione effettuare il training
            with open(modelPath + "model.json", 'r') as json_file:
                architecture = json_file.readlines()
                loaded_model = tf.keras.models.model_from_json(architecture[0])
                def closure(model):
                    def closureInside():
                        return model
                    return closureInside
                modelsAndWeights.append((closure(loaded_model),[weightsFileName]))
        predictions = testModels(modelsAndWeights,
                                testData,
                                useTPU,
                                tpu_address,
                                verbose = 0
                            )  

        for index in range(len(predictions)):
            confusionMatrix = predictionEvaluation([predictions[index]], labels, [0], topN = 1)
            print(ColoramaStyle.BRIGHT, "Modello ", modelsName[index],"] : pesi caricati ",modelsAndWeights[index][1], ColoramaStyle.RESET_ALL, sep="")
            confusionMatrixStat(confusionMatrix[0], printStat = True)
            if printPlot == True:
                plotConfusionMatrix(confusionMatrix[0], "TOP 1")
        
        joinModelButton = Button(description="Join")
        joinModelButton.on_click(onJoinClicked)
        joinCheckboxes = [widgets.Checkbox(description = modelName, value=True, width=width) for modelName in modelsName]
        joinGrid = widgets.GridBox(joinCheckboxes, layout=widgets.Layout(grid_template_columns="repeat("+str(repeat)+", "+str(gridWidth)+"px)",
                                                                grid_template_rows="repeat(auto-fill, "+str(gridHeight)+"px)"))
        display(AppLayout(header=joinGrid,
            footer=joinModelButton),output)

def onJoinClicked(button):
    global predictions
    global modelsName
    global joinCheckboxes
    with output:
        modelsIndexToJoin = [ value for value in [index * int(joinCheckboxes[index].value == True) + int(joinCheckboxes[index].value == False) * -1000 for index in range(len(modelsName))] if value >= 0]
        print(modelsIndexToJoin)
        confusionMatrix = predictionEvaluation(predictions, labels, modelsIndexToJoin, topN = 1)
        confusionMatrixStat(confusionMatrix[0], printStat = True)
        if printPlot == True:
            plotConfusionMatrix(confusionMatrix[0], "TOP 1")

models = list(filter(lambda name : os.path.isdir(os.path.join(trainingPath,name)),os.listdir(trainingPath)))
models.sort(key=sortByTime)

checkboxes = [widgets.Checkbox(description = modelName, value=False, width=width) for modelName in models]
if autoSelectLast:
    checkboxes[-1].value = True

grid = widgets.GridBox(checkboxes, layout=widgets.Layout(grid_template_columns="repeat("+str(repeat)+", "+str(gridWidth)+"px)",
                                                        grid_template_rows="repeat(auto-fill, "+str(gridHeight)+"px)"))

button = Button(description="Update")
button.on_click(onUpdateClicked)

display(AppLayout(header=grid,
         footer=button),output)